In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=207201b3559a18ff98f1f56302ca3cbc39256472f353d9dc8ebbec63f3b8c81f
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
env: KAGGLE_USERNAME=olaadel
env: KAGGLE_KEY=66fb44d480b787b525f6ac8060e8f5b0
 99% 715M/724M [00:09<00:00, 86.6MB/s]
100% 724M/724M [00:09<00:00, 83.6MB/s]
Archive:  /content/fcis-sc-deeplearning-competition.zip
  inflating: submit.csv              
  inflating: test/test/003_1b.jpg    
  inflating: test/test/0310_piscines_8.jpg  
  inflating: test/test/051011Greenhouse.jpg  
  inflating: test/test/100_1238.jpg  
  inflating: test/test/1164876979_0.jpg  
  inflating: test/test/1208_hall_entrada_010.jpg  
  inflating: test/test/12_t1_H_S_Pool_Inside.jpg  
  inflating: test/test/13_2o_andar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip  '/content/drive/My Drive/test.zip'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json
import tensorflow as tf
import PIL
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np
import os
import pandas as pd


TEST_DIR = '/content/test/test'
TRAIN_DIR = '/content/train/train'

train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size=32,
    class_mode='categorical',
    target_size=(224,224))

conv_base = ResNet50(
    include_top=False,
    weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(10, activation='softmax')(x)
model = Model(conv_base.input, predictions)

optimizer = keras.optimizers.Adam()
from keras.optimizers import SGD
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(generator=train_generator,
                              samples_per_epoch = 3138 // 32,  # added in Kaggle
                              epochs = 20,
                              validation_data=0,
                              validation_steps=10  # added in Kaggle
                             )



testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    preds = model.predict(x)
    print(imagep, preds, np.argmax(preds) + 1)
    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("subm1.csv")


print('end')

Found 3138 images belonging to 10 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<keras_pre..., epochs=20, validation_data=0, validation_steps=10, steps_per_epoch=3)`


Epoch 1/20
3/3 [==============================] - 31s 10s/step - loss: 2.1774 - acc: 0.2396
Epoch 2/20
3/3 [==============================] - 24s 8s/step - loss: 1.7925 - acc: 0.4062
Epoch 3/20
3/3 [==============================] - 24s 8s/step - loss: 1.4828 - acc: 0.5208
Epoch 4/20
3/3 [==============================] - 24s 8s/step - loss: 1.3282 - acc: 0.5521
Epoch 5/20
3/3 [==============================] - 24s 8s/step - loss: 1.0851 - acc: 0.6458
Epoch 6/20
3/3 [==============================] - 24s 8s/step - loss: 1.0695 - acc: 0.6562
Epoch 7/20
3/3 [==============================] - 24s 8s/step - loss: 0.8329 - acc: 0.7812
Epoch 8/20
3/3 [==============================] - 24s 8s/step - loss: 1.0032 - acc: 0.6354
Epoch 9/20
3/3 [==============================] - 24s 8s/step - loss: 0.8226 - acc: 0.7396
Epoch 10/20
3/3 [==============================] - 17s 6s/step - loss: 0.9599 - acc: 0.7320
Epoch 11/20
3/3 [==============================] - 24s 8s/step - loss: 0.8762 - acc: 0.7